In [1]:
#for data processing
import pandas as pd
import numpy as np
import string
import re

#for plotting
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#for nlp
from collections import Counter
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer

from keras.preprocessing import sequence
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from sklearn import preprocessing
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
np.random.seed(0)
from keras.optimizers import RMSprop
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import os
import math

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akank\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akank\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
#load train data
cols = ['id', 'text', 'label', 'intensity']

anger_train = pd.read_csv("C:/projects/Train/anger-ratings-0to1.train.txt", header=None, sep='\t', names=cols, index_col=0)
fear_train = pd.read_csv("C:/projects/Train/fear-ratings-0to1.train.txt", header=None, sep='\t', names=cols, index_col=0)
sad_train = pd.read_csv("C:/projects/Train/joy-ratings-0to1.train.txt", header=None, sep='\t', names=cols, index_col=0)
joy_train = pd.read_csv("C:/projects/Train/sadness-ratings-0to1.train.txt", header=None, sep='\t', names=cols, index_col=0)

In [3]:
#load dev data
cols = ['id', 'text', 'label', 'intensity']

anger_test = pd.read_csv("C:/projects/Validation/anger-ratings-0to1.dev.gold.txt", header=None, sep='\t', names=cols, index_col=0)
fear_test = pd.read_csv("C:/projects/Validation/fear-ratings-0to1.dev.gold.txt", header=None, sep='\t', names=cols, index_col=0)
sad_test = pd.read_csv("C:/projects/Validation/joy-ratings-0to1.dev.gold.txt", header=None, sep='\t', names=cols, index_col=0)
joy_test = pd.read_csv("C:/projects/Validation/sadness-ratings-0to1.dev.gold.txt", header=None, sep='\t', names=cols, index_col=0)

In [4]:
anger_train.head(2)

,text,label,intensity
id,,,
10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938
10001,So my Indian Uber driver just called someone t...,anger,0.896


In [5]:
train_df = pd.concat([anger_train, fear_train, sad_train, joy_train])

In [6]:
test_df = pd.concat([anger_test, fear_test, sad_test, joy_test])

In [7]:
import re

class TextCleaner:

    def __init__(self, text: str):
        self.text = text

    def fully_clean(self):
        return self \
            .remove_urls() \
            .remove_mentions() \
            .remove_hashtags() \
            .remove_reserved_words() \
            .remove_punctuation() \
            .remove_single_letter_words() \
            .remove_blank_spaces() \
            .remove_stopwords() \
            .remove_numbers() \
            .remove_non_ascii() \
            .lowercase()

    def remove_urls(self):
        url_pattern = re.compile(
            r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
            r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')
        self.text = re.sub(pattern=url_pattern, repl='', string=self.text)
        return self

    def remove_punctuation(self):
        self.text = self.text.translate(str.maketrans('', '', string.punctuation))
        return self

    def remove_mentions(self):
        mentions_pattern = re.compile(r'@\w*')
        self.text = re.sub(pattern=mentions_pattern, repl='', string=self.text)
        return self

    def remove_hashtags(self):
        hashtags_pattern = re.compile(r'#\w*')
        self.text = re.sub(pattern=hashtags_pattern, repl='', string=self.text)
        return self

    def remove_reserved_words(self):
        reserved_words_pattern = re.compile(r'(RT|rt|FAV|fav|VIA|via)')
        self.text = re.sub(pattern=reserved_words_pattern, repl='', string=self.text)
        return self

    def remove_single_letter_words(self):
        single_letter_words_pattern = re.compile(r'(?<![\w\-])\w(?![\w\-])')
        self.text = re.sub(pattern=single_letter_words_pattern, repl='', string=self.text)
        return self

    def remove_blank_spaces(self):
        blank_spaces_pattern = re.compile(r'\s{2,}|\t')
        self.text = re.sub(pattern=blank_spaces_pattern, repl=' ', string=self.text)
        return self

    def remove_stopwords(self, extra_stopwords=None):
        if extra_stopwords is None:
            extra_stopwords = []
        tokenizer = TweetTokenizer()
        words = tokenizer.tokenize(self.text)
        stop_words = set(stopwords.words('english'))

        new_sentence = [w for w in words if w.lower() not in stop_words and w.lower() not in extra_stopwords]
        self.text = ' '.join(new_sentence)
        return self

    def remove_numbers(self, preserve_years=False):
        text_list = self.text.split(' ')
        for text in text_list[:]:
            if text.isnumeric():
                if preserve_years:
                    if not is_year(text):
                        text_list.remove(text)
                else:
                    text_list.remove(text)

        self.text = ' '.join(text_list)
        return self
    
    def is_year(self, text):
        try:
            year = int(text)
            return 1000 <= year <= 9999
        except ValueError:
            return False
    
    def remove_non_ascii(self):
        self.text = ''.join(char for char in self.text if ord(char) < 128)
        return self

    def lowercase(self):
        self.text = self.text.lower()
        return self



In [8]:
# Import TweetTokenizer
from nltk.tokenize import TweetTokenizer

# Rest of your code
text = train_df['text']
clean_text = []

for text_entry in text:
    c = TextCleaner(text_entry)
    c.fully_clean()
    cleaned_text = c.text
    clean_text.append(cleaned_text)

train_df['clean_text'] = clean_text 
train_df.head(5)

,text,label,intensity,clean_text
id,,,,
10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938,fuk heck moved fridge knock landlord door
10001,So my Indian Uber driver just called someone t...,anger,0.896,indian uber driver called someone word wasnt m...
10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896,asked parcel delivered pick store address
10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896,ef whichever butt wipe pulled fire alarm davis...
10004,Don't join @BTCare they put the phone down on ...,anger,0.896,dont join put phone talk rude taking money acc...


In [9]:
from nltk.tokenize import TweetTokenizer

# Rest of your code
text = test_df['text']
clean_text = []

for text_entry in text:
    c = TextCleaner(text_entry)
    c.fully_clean()
    cleaned_text = c.text
    clean_text.append(cleaned_text)

test_df['clean_text'] = clean_text 
test_df.head(5)

,text,label,intensity,clean_text
id,,,,
10857,@ZubairSabirPTI pls dont insult the word 'Molna',anger,0.479,pls dont insult word molna
10858,@ArcticFantasy I would have almost took offens...,anger,0.458,would almost took offense actually snapped
10859,@IllinoisLoyalty that Rutgers game was an abom...,anger,0.562,rutgers game abomination affront god man must ...
10860,@CozanGaming that's what lisa asked before she...,anger,0.500,thats lisa asked staed raging call heh
10861,Sometimes I get mad over something so minuscul...,anger,0.708,sometimes get mad something minuscule try ruin...


In [10]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import SpatialDropout1D, Dropout, LeakyReLU
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.regularizers import l2

# # Assuming your 'intensity' values are continuous, not categorical
# intensity_values = train_df['intensity'].values
# test_intensity_values = test_df['intensity'].values

# Tokenize the text data
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(train_df['clean_text'].values)  # Corrected line

X = tokenizer.texts_to_sequences(train_df['clean_text'].values)
X = pad_sequences(X)


# Convert labels to regression values
intensity_values = train_df['intensity'].values

# Preprocess test data
test_sequences = tokenizer.texts_to_sequences(test_df['clean_text'].values)
test_data_padded = pad_sequences(test_sequences, maxlen=X.shape[1])

# Convert test labels to regression values
intensity_values_test = test_df['intensity'].values

# Model Building- best combo tried
model = Sequential()
model.add(Embedding(3666, 120, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear')) 


# 1 neuron for regression task
#1st change lr is 0.01,early stopping patience=5

# Compile the model 
lower_learning_rate = 0.01
optimizer = Adam(learning_rate=lower_learning_rate)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_absolute_error'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('bestmodel_weights.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
model.fit(
    X, intensity_values,
    epochs=100,
    batch_size=16,
    verbose=1,
    validation_data=(test_data_padded, intensity_values_test),
    callbacks=[checkpoint,early_stopping]
)

Epoch 1/100
226/226 [==============================] - ETA: 0s - loss: 0.0537 - mean_absolute_error: 0.1809
Epoch 1: val_loss improved from inf to 0.03482, saving model to bestmodel_weights.h5
226/226 [==============================] - 44s 131ms/step - loss: 0.0537 - mean_absolute_error: 0.1809 - val_loss: 0.0348 - val_mean_absolute_error: 0.1502


C:\Users\akank\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
226/226 [==============================] - ETA: 0s - loss: 0.0359 - mean_absolute_error: 0.1531
Epoch 2: val_loss improved from 0.03482 to 0.03186, saving model to bestmodel_weights.h5
226/226 [==============================] - 28s 125ms/step - loss: 0.0359 - mean_absolute_error: 0.1531 - val_loss: 0.0319 - val_mean_absolute_error: 0.1444
Epoch 3/100
226/226 [==============================] - ETA: 0s - loss: 0.0331 - mean_absolute_error: 0.1472
Epoch 3: val_loss did not improve from 0.03186
226/226 [==============================] - 29s 126ms/step - loss: 0.0331 - mean_absolute_error: 0.1472 - val_loss: 0.0322 - val_mean_absolute_error: 0.1448
Epoch 4/100
226/226 [==============================] - ETA: 0s - loss: 0.0327 - mean_absolute_error: 0.1455
Epoch 4: val_loss did not improve from 0.03186
226/226 [==============================] - 29s 128ms/step - loss: 0.0327 - mean_absolute_error: 0.1455 - val_loss: 0.0346 - val_mean_absolute_error: 0.1491
Epoch 5/100
226/226 [====

In [11]:
#load test data
cols = ['id', 'text', 'label', 'intensity']

anger_testtarget = pd.read_csv("C:/projects/Test/anger-ratings-0to1.dev.target.txt", header=None, sep='\t', names=cols, index_col=0)
fear_testtarget = pd.read_csv("C:/projects/Test/fear-ratings-0to1.dev.target.txt", header=None, sep='\t', names=cols, index_col=0)
sad_testtarget = pd.read_csv("C:/projects/Test/joy-ratings-0to1.dev.target.txt", header=None, sep='\t', names=cols, index_col=0)
joy_testtarget = pd.read_csv("C:/projects/Test/sadness-ratings-0to1.dev.target.txt", header=None, sep='\t', names=cols, index_col=0)

In [12]:
testtarget_df = pd.concat([anger_testtarget, fear_testtarget, sad_testtarget, joy_testtarget])

In [13]:
# Import TweetTokenizer
from nltk.tokenize import TweetTokenizer

# Rest of your code
text = testtarget_df['text']
clean_text = []

for text_entry in text:
    c = TextCleaner(text_entry)
    c.fully_clean()
    cleaned_text = c.text
    clean_text.append(cleaned_text)

testtarget_df['clean_text'] = clean_text 
testtarget_df.head(5)

,text,label,intensity,clean_text
id,,,,
10857,@ZubairSabirPTI pls dont insult the word 'Molna',anger,NONE,pls dont insult word molna
10858,@ArcticFantasy I would have almost took offens...,anger,NONE,would almost took offense actually snapped
10859,@IllinoisLoyalty that Rutgers game was an abom...,anger,NONE,rutgers game abomination affront god man must ...
10860,@CozanGaming that's what lisa asked before she...,anger,NONE,thats lisa asked staed raging call heh
10861,Sometimes I get mad over something so minuscul...,anger,NONE,sometimes get mad something minuscule try ruin...


In [14]:
from keras.models import load_model
# Load the model with the best weights
best_model = load_model('model_weights_lr_changes.h5')

# Tokenize the test text data using the same tokenizer used for training
test_sequences = tokenizer.texts_to_sequences(testtarget_df['clean_text'].values)
test_data_padded = pad_sequences(test_sequences, maxlen=X.shape[1])

# Make predictions on the test data
predictions = best_model.predict(test_data_padded)

# Print or use the predictions as needed
print(predictions)

11/11 [==============================] - 1s 24ms/step
[[0.44208446]
 [0.37340182]
 [0.45451653]
 [0.46709263]
 [0.5053781 ]
 [0.5053781 ]
 [0.4636131 ]
 [0.47892603]
 [0.49047592]
 [0.49047592]
 [0.4101616 ]
 [0.53668195]
 [0.4292692 ]
 [0.5309336 ]
 [0.5309336 ]
 [0.4955299 ]
 [0.4955299 ]
 [0.45588386]
 [0.5298224 ]
 [0.49648744]
 [0.49878255]
 [0.618296  ]
 [0.42737263]
 [0.42737263]
 [0.43174595]
 [0.48799714]
 [0.5025647 ]
 [0.46430764]
 [0.49062616]
 [0.40991926]
 [0.61949456]
 [0.448307  ]
 [0.5076314 ]
 [0.5043791 ]
 [0.44730324]
 [0.51958054]
 [0.51958054]
 [0.4515415 ]
 [0.4574745 ]
 [0.41212043]
 [0.5389035 ]
 [0.5000509 ]
 [0.46662056]
 [0.46662056]
 [0.43845174]
 [0.44023347]
 [0.4049279 ]
 [0.40483302]
 [0.5446663 ]
 [0.4671961 ]
 [0.4671961 ]
 [0.53951234]
 [0.439587  ]
 [0.48484844]
 [0.48484844]
 [0.4243577 ]
 [0.43931678]
 [0.33842757]
 [0.42320567]
 [0.43799347]
 [0.3906824 ]
 [0.44506106]
 [0.434919  ]
 [0.434919  ]
 [0.4388469 ]
 [0.43825185]
 [0.41303092]
 [0.5371

In [15]:
# Assuming your test data is stored in a DataFrame called test_df
testtarget_df['predicted_intensity'] = predictions.flatten()

# Print or display the DataFrame with input test data and predicted intensity
print(testtarget_df[['clean_text', 'intensity', 'predicted_intensity']])

                                              clean_text intensity  \
id                                                                   
10857                         pls dont insult word molna      NONE   
10858         would almost took offense actually snapped      NONE   
10859  rutgers game abomination affront god man must ...      NONE   
10860             thats lisa asked staed raging call heh      NONE   
10861  sometimes get mad something minuscule try ruin...      NONE   
...                                                  ...       ...   
40855  common app randomly logged writing last pa col...      NONE   
40856  id rather laugh rarest genius beautiful allian...      NONE   
40857                    new stop asking invest new film      NONE   
40858  watched django unchained people may frown titt...      NONE   
40859  depressing despicable trump policies campaigni...      NONE   

       predicted_intensity  
id                          
10857             0.442084  
10

In [16]:
from scipy.stats import pearsonr, spearmanr

# Assuming you have predictions for the test set
predictions = model.predict(test_data_padded)

# Calculate Pearson correlation
pearson_corr, _ = pearsonr(predictions.flatten(), intensity_values_test)
print(f'Pearson Correlation: {pearson_corr:.4f}')

# Calculate Spearman correlation
spearman_corr, _ = spearmanr(predictions.flatten(), intensity_values_test)
print(f'Spearman Correlation: {spearman_corr:.4f}')

11/11 [==============================] - 1s 25ms/step
Pearson Correlation: 0.3033
Spearman Correlation: 0.2876


In [17]:
testtarget_df[['text','clean_text','predicted_intensity']].to_csv('LSTM_output_3666.csv', index=False)

In [18]:
testtarget_df[['text','clean_text','predicted_intensity']].to_csv('LSTM_output_3666.txt', index=False, sep='\t')